# CMS Open Data $t\bar{t}$-analysis using RDF
This notebook was developed to showcase [RDF](https://root.cern/doc/master/classROOT_1_1RDataFrame.html)
applying to [2015 CMS Open Data](https://cms.cern/news/first-cms-open-data-lhc-run-2-released) $t\bar{t}$-analysis.\
This is a technical implementation of analysis procedure presented [here](https://github.com/andriiknu/RDF/blob/master/doc.ipynb).\
There is [another implementation](https://github.com/iris-hep/analysis-grand-challenge/blob/main/analyses/cms-open-data-ttbar/coffea.ipynb) producing the same results with using [Coffea analysis framework]((https://github.com/iris-hep/analysis-grand-challenge/blob/main/analyses/cms-open-data-ttbar/coffea.ipynb)

### Setting up environment: imports

In [1]:
import json
import ROOT
from ROOT import RDataFrame, TCanvas, THStack
ROOT.EnableImplicitMT()
import numpy as np
%jsroot on
print(ROOT.GetThreadPoolSize())
# ROOT.TH1.SetDefaultSumw2(true)
verbosity = ROOT.Experimental.RLogScopedVerbosity(ROOT.Detail.RDF.RDFLogChannel(), ROOT.Experimental.ELogLevel.kInfo)

Welcome to JupyROOT 6.26/04
10


### Setting up environment: ***helper.cpp*** compilation

In [2]:
ROOT.gSystem.CompileMacro("helper.cpp", "kO")

1

Info in <TUnixSystem::ACLiC>: creating shared library /eos/home-a/afalko/analysis-grand-challenge/analyses/cms-open-data-ttbar/helper_cpp.so


Here are a couple of functions used in analysis procedure.\
Some of them gives scaling factors applied for varyations construction:
* **pt_scale_up**: jet transverse momentum values scaling
* **t_res_up**: jet transverse momentum values scaling according to normal distribution
* **btag_weight_variation**: weights variations
* **flat_variation**: weights flat variations some of the processes

Other of them are C++ helpful functions for ROOT histograms handling.

In [3]:
N_FILES_MAX_PER_SAMPLE = 50
FILE = 'rdfseed050.root'

### TtbarAnalysis class
Encapsulates full analysis pipeline and provides helpful methods for data delivery and processing, and histogram construction. Serves as analysis manager.

In [4]:
from urllib.request import urlretrieve
import os


# ROOT.gInterpreter.Declare(
# '''
# float GetRandom(int seed=0)
# {
#     TRandom2 rnmd;
#     rnmd.SetSeed(seed);
#     return rnmd.Gaus(1,0.05);
# }
# '''
# )


class TtbarAnalysis(dict):

    def __init__(self, n_files_max_per_sample = 1, num_bins=25, bin_low = 50, bin_high = 550, download_input_data=False, use_local_data=False):
        
        self.variations = {}
        self.download_input_data = download_input_data  #set True to download input files
        self.use_local_data = use_local_data            #set True to use locally placed input files instead of https accessing
        self._nevts_total = {}
        self.n_files_max_per_sample = n_files_max_per_sample  #the number of files to be processed per sample
        self.input_data = self._construct_fileset()
        self.num_bins = num_bins
        self.bin_low = bin_low
        self.bin_high = bin_high
        self.xsec_info = {
            
            "ttbar": 396.87 + 332.97, # nonallhad + allhad, keep same x-sec for all
            "single_top_s_chan": 2.0268 + 1.2676,
            "single_top_t_chan": (36.993 + 22.175)/0.252,  # scale from lepton filter to inclusive
            "single_top_tW": 37.936 + 37.906,
            "wjets": 61457 * 0.252,  # e/mu+nu final states
            "data": None
        }
    
        

    def _construct_fileset(self):
        n_files_max_per_sample = self.n_files_max_per_sample
        with open ('ntuples.json') as f:
            file_info = json.load(f)
        fileset = {}
        for process in file_info.keys():
            if process == "data":
                continue  # skip data
            fileset[process] = {}
            self[process] = {}
            self._nevts_total[process] = {}
            for variation in file_info[process].keys():
#                 if variation != 'nominal': continue      
                file_list = file_info[process][variation]["files"]
                if n_files_max_per_sample != -1:
                    file_list = file_list[:n_files_max_per_sample]  # use partial set of samples
                file_paths = [f["path"] for f in file_list]
                nevts_total = sum([f["nevts"] for f in file_list])
                self._nevts_total[process].update({variation:nevts_total})
                fileset[process].update({variation: file_paths})
                if (self.download_input_data):
                    dir_name = f"input/{process}_{variation}"
                    os.makedirs(dir_name, exist_ok=True)
                    for i in range(len(file_paths)):
                        path = file_paths[i]
                        file = f"{dir_name}/{i}.root"
                        if not os.path.exists(file):
                            urlretrieve(path, file)   
                            print(f"{file} has been created")
                        else:
                            print(f"{file} already exists")
                self[process][variation] = {}
                
        return fileset

    def fill(self, process, variation, ):

        
        input_data = [f"root://eosuser.cern.ch//eos/user/a/afalko/analysis-grand-challenge/analyses/cms-open-data-ttbar/input/{process}_{variation}/{i}.root" for i in range(self.n_files_max_per_sample)] if self.use_local_data else self.input_data[process][variation]               
        d = RDataFrame('events', input_data)
        
        # normalization for MC
        x_sec = self.xsec_info[process]
        nevts_total = self._nevts_total[process][variation]
        lumi = 3378 # /pb
        xsec_weight = x_sec * lumi / nevts_total
        d = d.Define('weights', str(xsec_weight)) 
        
        if variation == 'nominal':
#             d = d.Define('res_up', 'ROOT::RVecF f(jet_pt.size()); for (auto& e:f) e = GetRandom(); return f;')
            d = d.Vary('jet_pt', "ROOT::RVec<ROOT::RVecF>{jet_pt*pt_scale_up(), jet_pt*pt_res_up(jet_pt)}", ["pt_scale_up", "pt_res_up"])
            if process == 'wjets':
                d = d.Vary('weights', 
                           "weights*flat_variation()",
                           [f"scale_var_{direction}" for direction in ["up", "down"]]
                          )
        ### event selection and filtering
        d = d.Define('electron_pt_mask', 'electron_pt>25').Define('muon_pt_mask', 'muon_pt>25').Define('jet_pt_mask', 'jet_pt>25')\
             .Filter('Sum(electron_pt_mask) + Sum(muon_pt_mask) == 1')\
             .Filter('Sum(jet_pt_mask) >= 4')\
             .Filter('Sum(jet_btag[jet_pt_mask]>=0.5)>=1')
             
        
        d = d.Vary('weights', 
                   'ROOT::RVecD{weights*btag_weight_variation(jet_pt[jet_pt_mask])}',
                   [f"{weight_name}_{direction}" for weight_name in [f"btag_var_{i}" for i in range(4)] for direction in ["up", "down"]]
                  ) if variation == 'nominal' else d
        
        
        measured = {"4j1b": "HT", "4j2b": 'trijet_mass'}
        for region in ["4j1b","4j2b"]:
            meas = measured[region]
            if region == "4j1b":

                fork = d.Filter('Sum(jet_btag[jet_pt_mask]>=0.5)==1').Define(meas, 'Sum(jet_pt[jet_pt_mask])')      

            elif region == "4j2b":

                fork = d.Filter('Sum(jet_btag[jet_pt_mask]>=0.5)>1').Define("jet_p4", 
                    "ROOT::VecOps::Construct<ROOT::Math::PxPyPzMVector>(jet_px[jet_pt_mask], jet_py[jet_pt_mask], jet_pz[jet_pt_mask], jet_mass[jet_pt_mask])"
                )

                fork = fork.Define('trijet', 
                    'ROOT::VecOps::Combinations(jet_pt[jet_pt_mask],3)'
                ).Define('ntrijet', 'trijet[0].size()')

                fork = fork.Define('trijet_p4', 
                                      'ROOT::VecOps::RVec<ROOT::Math::PxPyPzMVector> trijet_p4(ntrijet);'              +\
                                      'for (int i = 0; i < ntrijet; ++i) {'                                            +\
                                          'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'       +\
                                          'trijet_p4[i] = jet_p4[j1] + jet_p4[j2] + jet_p4[j3];'                       +\
                                      '}'                                                                              +\
                                      'return trijet_p4;'                                                                                                                          
                                     )

                #TODO  implement references
                fork = fork.Define('trijet_pt', 
                        'return ROOT::VecOps::Map(trijet_p4, [](ROOT::Math::PxPyPzMVector v) { return v.Pt(); })'
                                            )

                fork = fork.Define('trijet_btag', 
                                                  'ROOT::VecOps::RVec<bool> btag(ntrijet);'                                   +\
                                                  'for (int i = 0; i < ntrijet; ++i) {'                                       +\
                                                   'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'     +\
                                                   'btag[i]=std::max({jet_btag[j1], jet_btag[j2], jet_btag[j3]})>0.5;'        +\
                                                  '}'                                                                         +\
                                                  'return btag;'
                                            )

                fork=fork.Define(meas,
                                                  'double mass;'+\
                                                  'double Pt = 0;'+\
                                                  'double indx = 0;'+\
                                                  'for (int i = 0; i < ntrijet; ++i) {'               +\
                                                  '    if ((Pt < trijet_pt[i]) && (trijet_btag[i])) {'+\
                                                  '        Pt = trijet_pt[i];'+\
                                                  '        indx=i;'+\
                                                  '    }'                                            +\
                                                  '}'                                                +\
                                                  'mass = trijet_p4[indx].M();'             +\
                                                  'return mass;'
                                                 )
                

            res = fork.Histo1D((f'{process}_{variation}_{region}', process, self.num_bins, self.bin_low, self.bin_high), meas, 'weights')
            self.hist.append(res)
            print(f'histogram {region}_{process}_{variation} has been created')
            if variation == 'nominal':
                self.variations[f"{process}__{region}"] = ROOT.RDF.Experimental.VariationsFor(res)
            else:
                self[process][variation][region] = res

    def Fill(self):
        self.hist = []
        for process in self:
            
            for variation in self.input_data[process]:
                self.fill(process=process, variation=variation)

    def Accumulate(self):
        ROOT.RDF.RunGraphs(self.hist)  
     
    def TransfToDict(self):
        for key in self.variations.keys():
            hist_map = self.variations[key]
            key = str(key).split('__')
            process = key[0]; region = key[1]
            for hist_name in hist_map.GetKeys():
                variation = 'nominal' if hist_name == 'nominal' else str(hist_name).split(':')[1]
                if variation not in self[process]: self[process][variation] = {}
                hist = hist_map[hist_name]
                if not isinstance(hist, ROOT.TH1D): hist = hist.GetValue()
                analysisManager[process][variation][region] = hist
        analysisManager.ExportJSON()
        
    def GetProcStack(self, region, variation='nominal'):
        return [self[process][variation][region] for process in self]
    
    def GetVarStack(self, region, process="ttbar"):
        return [self[process][variation][region] for variation in self[process]]
    
    def ExportJSON(self):
        data = {}
        for process in self:
            data[process] = {}
            for variation in self[process]:
                data[process][variation] = [region for region in self[process][variation]]
        with open('data.json', 'w') as f:
            json.dump(data, f)
                
                
                    

### Analysis manager configuration
To perform analysis one need to instantiate TtbarAnalysis object. TtbarAnalysis.__init__ method accepts arguments defining all further TtbarAnalysi analysis behavior.
Main settings:
* n_files_max_per_sample - the number of files wich will be processed per sample (1 by default)
* use_local_data - specifies 

In [5]:
analysisManager = TtbarAnalysis(download_input_data=True, use_local_data=True, n_files_max_per_sample = N_FILES_MAX_PER_SAMPLE)
analysisManager.input_data

input/ttbar_nominal/0.root already exists
input/ttbar_nominal/1.root already exists
input/ttbar_nominal/2.root already exists
input/ttbar_nominal/3.root already exists
input/ttbar_nominal/4.root already exists
input/ttbar_nominal/5.root already exists
input/ttbar_nominal/6.root already exists
input/ttbar_nominal/7.root already exists
input/ttbar_nominal/8.root already exists
input/ttbar_nominal/9.root already exists
input/ttbar_nominal/10.root already exists
input/ttbar_nominal/11.root already exists
input/ttbar_nominal/12.root already exists
input/ttbar_nominal/13.root already exists
input/ttbar_nominal/14.root already exists
input/ttbar_nominal/15.root already exists
input/ttbar_nominal/16.root already exists
input/ttbar_nominal/17.root already exists
input/ttbar_nominal/18.root already exists
input/ttbar_nominal/19.root already exists
input/ttbar_nominal/20.root already exists
input/ttbar_nominal/21.root already exists
input/ttbar_nominal/22.root already exists
input/ttbar_nominal/2

input/ttbar_ME_var/47.root already exists
input/ttbar_ME_var/48.root already exists
input/ttbar_ME_var/49.root already exists
input/ttbar_PS_var/0.root already exists
input/ttbar_PS_var/1.root already exists
input/ttbar_PS_var/2.root already exists
input/ttbar_PS_var/3.root already exists
input/ttbar_PS_var/4.root already exists
input/ttbar_PS_var/5.root already exists
input/ttbar_PS_var/6.root already exists
input/ttbar_PS_var/7.root already exists
input/ttbar_PS_var/8.root already exists
input/ttbar_PS_var/9.root already exists
input/ttbar_PS_var/10.root already exists
input/ttbar_PS_var/11.root already exists
input/ttbar_PS_var/12.root already exists
input/ttbar_PS_var/13.root already exists
input/ttbar_PS_var/14.root already exists
input/ttbar_PS_var/15.root already exists
input/ttbar_PS_var/16.root already exists
input/ttbar_PS_var/17.root already exists
input/ttbar_PS_var/18.root already exists
input/ttbar_PS_var/19.root already exists
input/ttbar_PS_var/20.root already exists
in

input/single_top_tW_nominal/41.root already exists
input/single_top_tW_nominal/42.root already exists
input/single_top_tW_nominal/43.root already exists
input/single_top_tW_nominal/44.root already exists
input/single_top_tW_nominal/45.root already exists
input/single_top_tW_nominal/46.root already exists
input/single_top_tW_nominal/47.root already exists
input/single_top_tW_nominal/48.root already exists
input/single_top_tW_nominal/49.root already exists
input/wjets_nominal/0.root already exists
input/wjets_nominal/1.root already exists
input/wjets_nominal/2.root already exists
input/wjets_nominal/3.root already exists
input/wjets_nominal/4.root already exists
input/wjets_nominal/5.root already exists
input/wjets_nominal/6.root already exists
input/wjets_nominal/7.root already exists
input/wjets_nominal/8.root already exists
input/wjets_nominal/9.root already exists
input/wjets_nominal/10.root already exists
input/wjets_nominal/11.root already exists
input/wjets_nominal/12.root already

{'ttbar': {'nominal': ['https://xrootd-local.unl.edu:1094//store/user/AGC/datasets/RunIIFall15MiniAODv2/TT_TuneCUETP8M1_13TeV-powheg-pythia8/MINIAODSIM//PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1/00000/00DF0A73-17C2-E511-B086-E41D2D08DE30.root',
   'https://xrootd-local.unl.edu:1094//store/user/AGC/datasets/RunIIFall15MiniAODv2/TT_TuneCUETP8M1_13TeV-powheg-pythia8/MINIAODSIM//PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1/00000/020D0AF1-4BC2-E511-BDFC-0026B95ADB18.root',
   'https://xrootd-local.unl.edu:1094//store/user/AGC/datasets/RunIIFall15MiniAODv2/TT_TuneCUETP8M1_13TeV-powheg-pythia8/MINIAODSIM//PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1/00000/02837459-03C2-E511-8EA2-002590A887AC.root',
   'https://xrootd-local.unl.edu:1094//store/user/AGC/datasets/RunIIFall15MiniAODv2/TT_TuneCUETP8M1_13TeV-powheg-pythia8/MINIAODSIM//PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1/00000/0400D1B1-4CC2-E511-AD14-AC853DA06A1A.root',
   'https://xrootd-local.unl.edu:1094//store

In [6]:
import time
t0 = time.time()
analysisManager.Fill()
t1 = time.time()
print(f"\npreprocessing took {round(t1 - t0,2)} seconds")
analysisManager.Accumulate()
t2 = time.time()
print(f"processing took {round(t2 - t1,2)} seconds")
print(f"execution took {round(t2 - t0,2)} seconds")

histogram 4j1b_ttbar_nominal has been created
histogram 4j2b_ttbar_nominal has been created
histogram 4j1b_ttbar_scaledown has been created
histogram 4j2b_ttbar_scaledown has been created
histogram 4j1b_ttbar_scaleup has been created
histogram 4j2b_ttbar_scaleup has been created
histogram 4j1b_ttbar_ME_var has been created
histogram 4j2b_ttbar_ME_var has been created
histogram 4j1b_ttbar_PS_var has been created
histogram 4j2b_ttbar_PS_var has been created
histogram 4j1b_single_top_s_chan_nominal has been created
histogram 4j2b_single_top_s_chan_nominal has been created
histogram 4j1b_single_top_t_chan_nominal has been created
histogram 4j2b_single_top_t_chan_nominal has been created
histogram 4j1b_single_top_tW_nominal has been created
histogram 4j2b_single_top_tW_nominal has been created
histogram 4j1b_wjets_nominal has been created
histogram 4j2b_wjets_nominal has been created

preprocessing took 17.06 seconds
processing took 66.23 seconds
execution took 83.29 seconds


Info in <[ROOT.RDF] Info /build/jenkins/workspace/lcg_release_pipeline/build/projects/ROOT-6.26.04/src/ROOT/6.26.04/tree/dataframe/src/RLoopManager.cxx:722 in void ROOT::Detail::RDF::RLoopManager::Jit()>: Just-in-time compilation phase completed in 5.966087 seconds.
Info in <[ROOT.RDF] Info /build/jenkins/workspace/lcg_release_pipeline/build/projects/ROOT-6.26.04/src/ROOT/6.26.04/tree/dataframe/src/RLoopManager.cxx:722 in void ROOT::Detail::RDF::RLoopManager::Jit()>: Just-in-time compilation phase completed in 3.682927 seconds.
Info in <[ROOT.RDF] Info /build/jenkins/workspace/lcg_release_pipeline/build/projects/ROOT-6.26.04/src/ROOT/6.26.04/tree/dataframe/src/RLoopManager.cxx:722 in void ROOT::Detail::RDF::RLoopManager::Jit()>: Just-in-time compilation phase completed in 0.801399 seconds.
Info in <[ROOT.RDF] Info /build/jenkins/workspace/lcg_release_pipeline/build/projects/ROOT-6.26.04/src/ROOT/6.26.04/tree/dataframe/src/RLoopManager.cxx:722 in void ROOT::Detail::RDF::RLoopManager::Ji

In [7]:
analysisManager.TransfToDict()
analysisManager['ttbar'].keys()

dict_keys(['nominal', 'scaledown', 'scaleup', 'ME_var', 'PS_var', 'pt_res_up', 'pt_scale_up', 'btag_var_0_down', 'btag_var_0_up', 'btag_var_1_down', 'btag_var_1_up', 'btag_var_2_down', 'btag_var_2_up', 'btag_var_3_down', 'btag_var_3_up'])

In [8]:
output = ROOT.TFile.Open(FILE, 'RECREATE')
for process in analysisManager:
    for variation in analysisManager[process]:
        for region in analysisManager[process][variation]:
            hist_name = f"{region}_{process}_{variation}" if variation != 'nominal' else f"{region}_{process}"
            hist = analysisManager[process][variation][region]
            if not isinstance(hist, ROOT.TH1D): hist = hist.GetValue() #this this a bag
            if hist.IsZombie(): raise TypeError(hist_name)
            hist_sliced = ROOT.Slice(hist, 120, 550)
            hist_binned = hist_sliced.Rebin(2, hist.GetTitle())
            output.WriteObject(hist_binned, hist_name)
output.Close()

Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential me

In [9]:
c = TCanvas('c', 'c', 600, 400) 
hlist = analysisManager.GetProcStack(region='4j2b')
hs = THStack('j4b1', '>=4 jets, 1 b-tag (RDF); H_{T} [GeV]')
for h in hlist:
    ptr = h.Rebin(2, h.GetTitle())
    hs.Add(ptr)
hs.Draw('hist pfc plc')
c.Draw()
x = hs.GetXaxis()
x.SetTitleOffset(1.5)
x.CenterTitle()
c.BuildLegend(0.65, 0.7, 0.9, 0.9)

Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.


In [10]:
freshstack = analysisManager.GetVarStack(region='4j1b')
hs = THStack('j4b1btag', 'all variations (RDF); H_{T} [GeV]')
for h in freshstack:
    ptr = h.Rebin(2, h.GetTitle())
    ptr.SetFillColor(0)
    ptr.SetLineWidth(1)
    hs.Add(ptr)
hs.Draw('hist nostack')
c.Draw()
x = hs.GetXaxis()
x.SetRangeUser(120, 500)
x.SetTitleOffset(1.5)
x.CenterTitle()
c.BuildLegend(0.65, 0.7, 0.9, 0.9)

Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nb

In [13]:
# from validation import get_mismatched

def get_values(h):
    return np.array([h.GetBinContent(i+1) for i in range (h.GetNbinsX())])
def get_variances(h):
    return np.array([h.GetBinError(i+1) for i in range (h.GetNbinsX())])

def match_histos (rdf_hist, coffea_hist, precision):
    rdf_values = get_values(rdf_hist)
    coffea_values = get_values(coffea_hist)
    rdf_values = np.round(rdf_values, precision)
    coffea_values = np.round(coffea_values, precision)
    mask = rdf_values == coffea_values
    return not (False in mask)

def get_deviations(rdf_hist, coffea_hist):
    rdf_values = get_values(rdf_hist)
    coffea_values = get_values(coffea_hist)
    deviations = np.zeros(len(rdf_values))
    for i in range(len(deviations)):
        rdf_value = rdf_values[i]
        coffea_value = coffea_values[i]
        deviations[i] = 0 if round(coffea_value, 5) == round(rdf_value, 5) == 0 else 100*abs(rdf_value-coffea_value)/coffea_value
#         deviations[i] = 100*abs(rdf_value-coffea_value)/coffea_value

    return deviations


def get_mismatched (rdf, coffea):
    mismatched = []
    rdf = ROOT.TFile.Open(rdf)
    coffea = ROOT.TFile.Open(coffea)
    with open('data.json', 'r') as f:
        data = json.load(f)
    for process in data:
        for variation in data[process]:
            for region in data[process][variation]:
                hist_name = f"{region}_{process}_{variation}" if variation != 'nominal' else f"{region}_{process}"
                rdf_hist = rdf.Get(hist_name)
                coffea_hist = coffea.Get(hist_name)
                if (rdf_hist and coffea_hist):
                    deviations = get_deviations(rdf_hist, coffea_hist)
                    i = np.argmax(deviations); dev = deviations[i]
                    variance = coffea_hist.GetBinError(int(i+1))
                    abs_dev = abs(rdf_hist.GetBinContent(int(i+1))-coffea_hist.GetBinContent(int(i+1)))
#                     if deviation > 20:
#                     if dev > 0.0001 and "res_up" not in hist_name:
#                         print(f"deviation={dev:.4f}%\t-\t{hist_name}")
                    if "res_up" in hist_name:
                        print(f"deviation={dev:.2f}% deviation/error={100*abs_dev/variance:.0f}%\t-\t{hist_name}({i} bin)")
                        mismatched.append(hist_name)
                else:
                    print(hist_name)
                    raise ValueError('rdf_hist and coffea_hist is Zombie')
    return mismatched

mism = get_mismatched('rdfseed050.root', 'histograms_local50.root')
#nothing == all right

deviation=3.65% deviation/error=113%	-	4j1b_ttbar_pt_res_up(0 bin)
deviation=0.54% deviation/error=56%	-	4j2b_ttbar_pt_res_up(7 bin)
deviation=3.98% deviation/error=118%	-	4j1b_single_top_s_chan_pt_res_up(0 bin)
deviation=2.57% deviation/error=95%	-	4j2b_single_top_s_chan_pt_res_up(8 bin)
deviation=2.18% deviation/error=87%	-	4j1b_single_top_t_chan_pt_res_up(10 bin)
deviation=1.25% deviation/error=70%	-	4j2b_single_top_t_chan_pt_res_up(9 bin)
deviation=0.81% deviation/error=71%	-	4j1b_single_top_tW_pt_res_up(9 bin)
deviation=0.42% deviation/error=33%	-	4j2b_single_top_tW_pt_res_up(7 bin)
deviation=4.40% deviation/error=120%	-	4j1b_wjets_pt_res_up(0 bin)
deviation=5.06% deviation/error=45%	-	4j2b_wjets_pt_res_up(7 bin)
